In [ ]:
## Here I am dividing the data first based onto protected attribute value and then train two separate models

import pandas as pd
import random,time
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import sys
sys.path.append(os.path.abspath('..'))

from Measure import measure_final_score,calculate_recall,calculate_far,calculate_precision,calculate_accuracy

In [ ]:
## Load dataset
dataset_orig = pd.read_csv('../dataset/compas-scores-two-years.csv')



## Drop categorical features
## Removed two duplicate coumns - 'decile_score','priors_count'
dataset_orig = dataset_orig.drop(['id','name','first','last','compas_screening_date','dob','age','juv_fel_count','decile_score','juv_misd_count','juv_other_count','days_b_screening_arrest','c_jail_in','c_jail_out','c_case_number','c_offense_date','c_arrest_date','c_days_from_compas','c_charge_desc','is_recid','r_case_number','r_charge_degree','r_days_from_arrest','r_offense_date','r_charge_desc','r_jail_in','r_jail_out','violent_recid','is_violent_recid','vr_case_number','vr_charge_degree','vr_offense_date','vr_charge_desc','type_of_assessment','decile_score','score_text','screening_date','v_type_of_assessment','v_decile_score','v_score_text','v_screening_date','in_custody','out_custody','start','end','event'],axis=1)

## Drop NULL values
dataset_orig = dataset_orig.dropna()


## Change symbolics to numerics
dataset_orig['sex'] = np.where(dataset_orig['sex'] == 'Female', 1, 0)
dataset_orig['race'] = np.where(dataset_orig['race'] != 'Caucasian', 0, 1)
dataset_orig['priors_count'] = np.where((dataset_orig['priors_count'] >= 1 ) & (dataset_orig['priors_count'] <= 3), 3, dataset_orig['priors_count'])
dataset_orig['priors_count'] = np.where(dataset_orig['priors_count'] > 3, 4, dataset_orig['priors_count'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == 'Greater than 45',45,dataset_orig['age_cat'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == '25 - 45', 25, dataset_orig['age_cat'])
dataset_orig['age_cat'] = np.where(dataset_orig['age_cat'] == 'Less than 25', 0, dataset_orig['age_cat'])
dataset_orig['c_charge_degree'] = np.where(dataset_orig['c_charge_degree'] == 'F', 1, 0)


## Rename class column
dataset_orig.rename(index=str, columns={"two_year_recid": "Probability"}, inplace=True)

## Here did not rec means 0 is the favorable lable
# dataset_orig['Probability'] = np.where(dataset_orig['Probability'] == 0, 1, 0)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
dataset_orig = pd.DataFrame(scaler.fit_transform(dataset_orig),columns = dataset_orig.columns)


# divide the data based on sex
dataset_orig_male , dataset_orig_female = [x for _, x in dataset_orig.groupby(dataset_orig['sex'] == 0)]

# divide the data based on race
dataset_orig_male_white , dataset_orig_male_black = [x for _, x in dataset_orig_male.groupby(dataset_orig['race'] == 0)]
dataset_orig_female_white , dataset_orig_female_black = [x for _, x in dataset_orig_female.groupby(dataset_orig['race'] == 0)]

print(dataset_orig_male_white.shape)
print(dataset_orig_male_black.shape)
print(dataset_orig_female_white.shape)
print(dataset_orig_female_black.shape)

# Train the model for male white

In [ ]:
dataset_orig_male_white['race'] = 0
dataset_orig_male_white['sex'] = 0
X_train, y_train = dataset_orig_male_white.loc[:, dataset_orig_male_white.columns != 'Probability'], dataset_orig_male_white['Probability']
# --- LSR
clf1 = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf1.fit(X_train, y_train)

# print(X_train_male['sex'])
import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_male_white.columns)-1)
plt.barh(y,clf1.coef_[0])
plt.yticks(y,dataset_orig_male_white.columns)
plt.show()

print(clf1.coef_[0])

# Train the model for male black

In [ ]:
dataset_orig_male_black['race'] = 0
dataset_orig_male_black['sex'] = 0
X_train, y_train = dataset_orig_male_black.loc[:, dataset_orig_male_black.columns != 'Probability'], dataset_orig_male_black['Probability']
# --- LSR
clf2 = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf2.fit(X_train, y_train)

# print(X_train_male['sex'])
import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_male_black.columns)-1)
plt.barh(y,clf2.coef_[0])
plt.yticks(y,dataset_orig_male_black.columns)
plt.show()

print(clf2.coef_[0])

# Train the model for female white

In [ ]:
dataset_orig_female_white['race'] = 0
dataset_orig_female_white['sex'] = 0
X_train, y_train = dataset_orig_female_white.loc[:, dataset_orig_female_white.columns != 'Probability'], dataset_orig_female_white['Probability']
# --- LSR
clf3 = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf3.fit(X_train, y_train)

# print(X_train_male['sex'])
import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_female_white.columns)-1)
plt.barh(y,clf3.coef_[0])
plt.yticks(y,dataset_orig_female_white.columns)
plt.show()

print(clf3.coef_[0])

# Train the model for female black

In [ ]:
dataset_orig_female_black['race'] = 0
dataset_orig_female_black['sex'] = 0
X_train, y_train = dataset_orig_female_black.loc[:, dataset_orig_female_black.columns != 'Probability'], dataset_orig_female_black['Probability']
# --- LSR
clf4 = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
clf4.fit(X_train, y_train)

# print(X_train_male['sex'])
import matplotlib.pyplot as plt
y = np.arange(len(dataset_orig_female_black.columns)-1)
plt.barh(y,clf4.coef_[0])
plt.yticks(y,dataset_orig_female_black.columns)
plt.show()

print(clf4.coef_[0])

# Remove biased rows

In [ ]:
print(dataset_orig.shape)

for index,row in dataset_orig.iterrows():    
    row = [row.values[0:len(row.values)-1]]
    y_male_white = clf1.predict(row)
    y_male_black = clf2.predict(row)
    y_female_white = clf3.predict(row)
    y_female_black = clf4.predict(row)
    if not((y_male_white[0] == y_male_black[0]) and (y_male_black[0] == y_female_white[0]) and (y_female_white[0] == y_female_black[0])):
        dataset_orig = dataset_orig.drop(index)
#     else:
#         print(y_male_white[0],y_male_black[0],y_female_white[0],y_female_black[0])
    
print(dataset_orig.shape)

# Train and test on unbiased data

In [ ]:
print(dataset_orig.shape)
np.random.seed(0)
## Divide into train,validation,test
dataset_orig_train, dataset_orig_test = train_test_split(dataset_orig, test_size=0.2, random_state=0,shuffle = True)

X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'Probability'], dataset_orig_train['Probability']
X_test , y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'Probability'], dataset_orig_test['Probability']

# --- LSR
clf = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=100)
# --- CART
# clf = tree.DecisionTreeClassifier()

# clf.fit(X_train, y_train)
# import matplotlib.pyplot as plt
# y = np.arange(len(dataset_orig_train.columns)-1)
# plt.barh(y,clf.coef_[0])
# plt.yticks(y,dataset_orig_train.columns)
# plt.show()

# print(clf_male.coef_[0])
# y_pred = clf.predict(X_test)
# cnf_matrix_test = confusion_matrix(y_test,y_pred)

# print(cnf_matrix_test)
# TN, FP, FN, TP = confusion_matrix(y_test,y_pred).ravel()


print("recall :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'recall'))
print("far :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'far'))
print("precision :", measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'precision'))
print("accuracy :",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'accuracy'))
print("aod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'aod'))
print("eod sex:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'race', 'eod'))

print("TPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'sex', 'TPR'))
print("FPR:",measure_final_score(dataset_orig_test, clf, X_train, y_train, X_test, y_test, 'sex', 'FPR'))